# Combine supervised and unsupervised learning

In [1]:
import pandas as pd
from utils import Packet, Flow

def get_flows(df: pd.DataFrame, key_type: str = "default") -> dict:
    mp = dict()
    for idx in range(len(df)): # simulate the process of packet processing
        row = df.iloc[idx]
        pkt = Packet(
            src_ip=row["src_ip"],
            dst_ip=row["dst_ip"],
            src_port=row["src_port"],
            dst_port=row["dst_port"],
            protocol=row["protocol"],
            proto_code=row["proto_code"],
            pkt_length=row["pkt_length"],
            timestamp=row["timestamp"],
            ttl=row["ttl"],
            tcp_window=row["tcp_window"],
            tcp_dataoffset=row["tcp_dataoffset"],
            udp_length=row["udp_length"],
            label=row["label"],
        )
        key = pkt.key(type=key_type)
        if key not in mp:
            mp[key] = Flow()
        mp[key].add_packet(pkt)
    return mp

In [2]:
import torch
from config import whisper_config

def transform(mp: dict, feature_type: str = "whisper", 
              data_type: str = "train", test_data_aug: bool = True):
    packet_data, flow_data = [], []
    packet_labels, flow_labels = [], []
    for key, flow in mp.items():
        vec = flow.vector(feature_type=feature_type)
        if feature_type == "whisper":
            if len(vec) <= (whisper_config["n_fft"] // 2):
                # packet level features
                # vec = flow.packet_vector(agg_type="mean") + flow.packet_vector(agg_type="std") \
                #     + flow.packet_vector(agg_type="max") + flow.packet_vector(agg_type="min")
                # packet_data.append(vec)
                # packet_labels.append(flow.label)

                # implement fft on short flows
                ten = torch.tensor(vec)
                ten_fft = torch.fft.fft(ten, n=(whisper_config["n_fft"] // 2)+1)
                ten_power = torch.pow(ten_fft.real, 2) + torch.pow(ten_fft.imag, 2)
                ten_res = (ten_power.squeeze()+1).log2()
                ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
                ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)
                if data_type == "test" and test_data_aug:
                    # data shape for test data augmentation: (n_flow, n_sample, floor(n_fft/2)+1)
                    packet_data.append([ten_res.tolist()])
                else:
                    # data shape for no data augmentation: (n_flow, floor(n_fft/2)+1)
                    packet_data.append(ten_res.tolist())
                packet_labels.append(flow.label)
                
            else:
                # flow level featrues
                ten = torch.tensor(vec)
                # stft requirement: input_size > (n_fft // 2)
                # default return shape: (floor(n_fft/2)+1, n_frame, 2)
                ten_fft = torch.stft(ten, whisper_config["n_fft"])
                ten_power = torch.pow(ten_fft[:,:,0], 2) + torch.pow(ten_fft[:,:,1], 2)
                ten_res = ((ten_power.squeeze()+1).log2()).permute(1,0)
                ten_res = torch.where(torch.isnan(ten_res), torch.zeros_like(ten_res), ten_res)
                ten_res = torch.where(torch.isinf(ten_res), torch.zeros_like(ten_res), ten_res)
                # ten_res shape: (n_frame, floor(n_fft/2)+1)
                if data_type == "train":
                    if (ten_res.size(0) > whisper_config["mean_win_train"]):
                        for _ in range(whisper_config["num_train_sample"]):
                            start_idx = torch.randint(0, ten_res.size(0)
                                        - whisper_config["mean_win_train"], (1,)).item()
                            ten_tmp = ten_res[start_idx:start_idx+whisper_config["mean_win_train"],:].mean(dim=0)
                            flow_data.append(ten_tmp.tolist())
                            flow_labels.append(flow.label)
                    else:
                        flow_data.append(ten_res.mean(dim=0).tolist())
                        flow_labels.append(flow.label)
                else: # for test
                    if test_data_aug:
                        tmp_data = []
                        if (ten_res.size(0) > whisper_config["mean_win_test"]):
                            # data augmentation for kmeans on flows with length > mean_win_test
                            for idx in range(0, ten_res.size(0) - whisper_config["mean_win_test"], 
                                            whisper_config["mean_win_test"]):
                                ten_tmp = ten_res[idx:idx+whisper_config["mean_win_test"],:].mean(dim=0)
                                tmp_data.append(ten_tmp.tolist())
                        else:
                            # no data augmentation for kmeans on flows with length < mean_win_test
                            tmp_data.append(ten_res.mean(dim=0).tolist())
                        flow_data.append(tmp_data)
                        # data shape for augmentation: (n_flow, n_sample, floor(n_fft/2)+1)
                    else: # for other detection methods
                        flow_data.append(ten_res.mean(dim=0).tolist())
                        # data shape for no augmentation: (n_flow, floor(n_fft/2)+1)
                    flow_labels.append(flow.label)
        elif feature_type == "encoding":
            # directly use the whisper encoding vector
            pass
        else: # for other feature types
            pass
    return packet_data, packet_labels, flow_data, flow_labels

/root/anaconda3/envs/PRO/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Mix benign and attack traffic

In [3]:
import numpy as np

def get_train_test_df(benign_path: str, attack_path: str):
    df_benign = pd.read_csv(benign_path)
    df_benign["label"] = 1
    df_attack = pd.read_csv(attack_path)
    df_attack["label"] = -1
    df_mix = pd.concat([df_benign, df_attack], ignore_index=True, axis=0)
    df_group = df_mix.groupby(["src_ip", "dst_ip", "src_port", "dst_port", "protocol"])
    df_mix["group"] = df_group.ngroup()
    df_mix["group"] = df_mix["group"] % 10
    df_mix["group"] = df_mix["group"].apply(lambda x: "train" if x < 6 else "test")
    # df_mix["group"].apply(lambda x: "train" if np.random.rand() < train_test_ratio else "test")
    df_train = df_mix[df_mix["group"] == "train"]
    df_test = df_mix[df_mix["group"] == "test"]
    return df_train, df_test

In [9]:
def get_mix_df(benign_path: str = None, 
                attack_path: str = None, 
                attack_ratio: int = None):
    assert benign_path is not None or attack_path is not None, \
        "benign_path and attack_path cannot be None at the same time"
    if benign_path is not None:
        df_benign = pd.read_csv(benign_path)
        df_benign["label"] = 1
    else:
        df_benign = None
    if attack_path is not None:
        df_attack = pd.read_csv(attack_path)
        df_attack["label"] = -1
        if attack_ratio is not None:
            df_group = df_attack.groupby(["src_ip", "dst_ip", "src_port", "dst_port", "protocol"])
            df_attack["group"] = df_group.ngroup()
            df_attack["group"] = df_attack["group"] % 10
            df_attack["group"] = df_attack["group"].apply(lambda x: "train" if x < attack_ratio else "test")
            df_attack = df_attack[df_attack["group"] == "train"]
    else:
        df_attack = None
    df_mix = pd.concat([df_benign, df_attack], ignore_index=True, axis=0)
    return df_mix

In [5]:
import os

benign_filenames = [os.path.join("train_set", "benign" + str(i) + ".csv") 
                    for i in range(1, 3)] + ["dataset_lite/mirai-benign.csv"]
attack_filenames = [os.path.join("dataset_lite", x+".csv") for x in 
                    ["BruteForce-Web", "BruteForce-XSS", "infiltration", 
                    "osscan", "SQL_Injection", "ssldosA10only", "mirai-attack"]]

In [6]:
from sklearn.tree import DecisionTreeClassifier
import os
import json
import skops.io as sio

def train(data, labels, save_path):
    clf = DecisionTreeClassifier(random_state=0)
    clf.fit(data, labels)
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    sio.dump(clf, save_path)

from sklearn.metrics import accuracy_score

def test(data, labels, load_path, test_data_aug=False):
    clf = sio.load(load_path, True)
    if not test_data_aug:
        preds = clf.predict(data)
    else:
        preds = []
        for val in data:
            pred = clf.predict(val)
        preds.append(1 if sum(pred) > -1*len(pred) else -1)
    return accuracy_score(labels, preds)

In [13]:
USE_SHORT_FLOW = True

train_benign_filename = "dataset/benign_small.csv"
train_attack_filename = attack_filenames[1]

df_train = get_mix_df(benign_path=train_benign_filename, 
                    attack_path=train_attack_filename,
                    attack_ratio=1)
train_packet_data, train_packet_labels, train_flow_data, train_flow_labels \
    = transform(get_flows(df_train), data_type="train")

basename = "all" if not USE_SHORT_FLOW else "long"
save_path = os.path.join("model", "whisper", "decision_tree", 
            os.path.basename(train_benign_filename) + "_" + \
            os.path.basename(train_attack_filename) + "_" + basename + ".skops")

train_data = train_flow_data if not USE_SHORT_FLOW else train_flow_data + train_packet_data
train_labels = train_flow_labels if not USE_SHORT_FLOW else train_flow_labels + train_packet_labels
train(train_data, train_labels, save_path)

In [14]:
USE_DATA_AUG = False

detect_type = "decision_tree" if USE_DATA_AUG else "decision_tree_no_aug"
accuracy_dict = {}

for test_benign_filename in benign_filenames:
    df_test = get_mix_df(benign_path=test_benign_filename)
    test_packet_data, test_packet_labels, test_flow_data, test_flow_labels \
    = transform(get_flows(df_train), data_type="test", test_data_aug=USE_DATA_AUG)
    test_data = test_flow_data if not USE_SHORT_FLOW else test_flow_data + test_packet_data
    test_labels = test_flow_labels if not USE_SHORT_FLOW else test_flow_labels + test_packet_labels
    acc = test(test_data, test_labels, save_path, test_data_aug=USE_DATA_AUG)
    print(f"accuracy of {test_benign_filename}: {acc}")
    accuracy_dict[test_benign_filename] = acc
    
for test_attack_filename in attack_filenames:
    df_test = get_mix_df(attack_path=test_attack_filename)
    test_packet_data, test_packet_labels, test_flow_data, test_flow_labels \
    = transform(get_flows(df_train), data_type="test", test_data_aug=USE_DATA_AUG)
    test_data = test_flow_data if not USE_SHORT_FLOW else test_flow_data + test_packet_data
    test_labels = test_flow_labels if not USE_SHORT_FLOW else test_flow_labels + test_packet_labels
    acc = test(test_data, test_labels, save_path, test_data_aug=USE_DATA_AUG)
    print(f"accuracy of {test_attack_filename}: {acc}")
    accuracy_dict[test_attack_filename] = acc

accuracy_base_name = "flow-accuracy.json" if not USE_SHORT_FLOW else "all-accuracy.json"
accuracy_save_path = os.path.join("result", "whisper", detect_type, 
                    os.path.basename(train_benign_filename)+"-"+os.path.basename(train_attack_filename),
                    accuracy_base_name)
os.makedirs(os.path.dirname(accuracy_save_path), exist_ok=True)
with open(accuracy_save_path, "w") as f:
    json.dump(accuracy_dict, f, indent=4)

accuracy of train_set/benign1.csv: 1.0
accuracy of train_set/benign2.csv: 1.0
